## Topics

- Runnable
- Prompt Template
- Sequential memory
- Output parsing

In [2]:
%pip install langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 5.8 MB/s eta 0:00:00a 0:00:01
  Using cached dataclasses_json-0.6.6-py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.7 MB/s eta 0:00:00
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-macosx_11_0_arm64.whl (389 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 27.4 MB/s eta 0:00:00
  Using cached multidict-6.0.5-cp310-cp310-macosx_11_0_arm64.whl (30 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached yarl-1.9.4-cp310-cp310-macosx_11_0_arm64.whl (79 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-macosx_11_0_arm64.whl (52 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl (49 k

In [3]:
import os
from langchain_openai import ChatOpenAI  # pip install langchain-openai

llm = ChatOpenAI(
    openai_api_key="", 
    temperature=.75, 
    max_tokens=1024, 
    request_timeout=30
)

In [4]:
llm.invoke("test")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f33b47f-269d-45f8-aea3-63b1a7671071-0')

In [7]:
from langchain.prompts import ChatPromptTemplate  # pip install langchain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Act as a world class Machine Learning engineer. Use english language. End your answers with a reference to the beauty of using data science in any decision you make."),
    ("user", "{input}")
])

# concatenazione del prompt al modello
chain = prompt | llm

## Runnable Interface

To simplify the creation of even very complex event/execution chains, all LangChain components implement a "runnable" protocol through a common interface that allows any component to be used in a standard way. Below are the three main methods:

* **stream** - send partial responses as they are generated
* **invoke** - execute the chain on a single input
* **batch** - execute the chain on multiple inputs

### Input and Output of Main Components
<img src="assets/componenti_io.png" width="600">

One of the advantages of Runnable interfaces is that runnable components can be chained together in execution sequences, allowing the output of one component to automatically become the input to another. The *pipe* command **|** is used for this purpose in LCEL (LangChain Expression Language), enabling the creation of runnable components from other runnable components by configuring them into a sequence that will work synergistically.


In [9]:
chain.invoke({"input": "hello!"})

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 45, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d89d5046-5052-44c6-b709-ca6d8c185289-0')

# ConversationBufferMemory

[`ConversationBufferMemory`](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html) is a tool in LangChain that helps keep track of a conversation. It stores the messages exchanged between the user and the AI so that the AI can remember what has been said earlier. This helps the AI maintain context and continuity in the conversation.

`ConversationBufferMemory` is a type of sequential memory in Langchain:

<img src="assets/sequential-memory.png" width="300" />


Here’s a basic example of how to add messages to a `ConversationBufferMemory` and how to get back the messages:

In [14]:
from langchain.memory import ConversationBufferMemory

# Create a new conversation memory
memory = ConversationBufferMemory()

# Add user and AI messages to the memory
memory.chat_memory.add_user_message("Hello")
memory.chat_memory.add_ai_message("Hi! How you doin'?")
memory.chat_memory.add_user_message("Fine, thanks.")

print(memory.load_memory_variables({})['history'])

Human: Hello
AI: Hi! How you doin'?
Human: Fine, thanks.


In [16]:
memory = ConversationBufferMemory(return_messages=True)

# Add user and AI messages to the memory
memory.chat_memory.add_user_message("Hello")
memory.chat_memory.add_ai_message("Hi! How you doin'?")
memory.chat_memory.add_user_message("Fine, thanks.")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello'),
  AIMessage(content="Hi! How you doin'?"),
  HumanMessage(content='Fine, thanks.')]}

# Introduction to PromptTemplate

The `PromptTemplate` is a powerful feature designed to streamline and standardize the creation of prompts for various applications, such as chatbots, automated responses, or data entry forms. It provides a structured format that can be reused across different scenarios, ensuring consistency and efficiency in how inputs are solicited and processed.



In [17]:
# dynamic template and use of a Memory Buffer

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:
{chat}

New question: {question}
Answer:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat")

from langchain.chains import LLMChain

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/Users/giumast/miniforge3/envs/tensorflow/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [18]:
conversation.invoke({"question": "Hello, i lake the orange color."})



> Entering new LLMChain chain...
Prompt after formatting:
Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:


New question: Hello, i lake the orange color.
Answer:

> Finished chain.


{'question': 'Hello, i lake the orange color.',
 'chat': '',
 'text': "Orange color is indeed beautiful! Did you know that data science can also be beautiful in the way it uncovers patterns and insights from complex datasets? Just like how different shades of orange can create a stunning visual impact, the analysis and visualization of data in data science can bring out the beauty of information hidden within the numbers. Let's explore the beauty of data science together!"}

In [22]:
print(memory.load_memory_variables({})['chat'])

Human: Hello, i lake the orange color.
AI: Orange color is indeed beautiful! Did you know that data science can also be beautiful in the way it uncovers patterns and insights from complex datasets? Just like how different shades of orange can create a stunning visual impact, the analysis and visualization of data in data science can bring out the beauty of information hidden within the numbers. Let's explore the beauty of data science together!


In [23]:
conversation.invoke({"question": "Tell me 3 fruits of my favourite color"})



> Entering new LLMChain chain...
Prompt after formatting:
Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:
Human: Hello, i lake the orange color.
AI: Orange color is indeed beautiful! Did you know that data science can also be beautiful in the way it uncovers patterns and insights from complex datasets? Just like how different shades of orange can create a stunning visual impact, the analysis and visualization of data in data science can bring out the beauty of information hidden within the numbers. Let's explore the beauty of data science together!

New question: Tell me 3 fruits of my favourite color
Answer:

> Finished chain.


{'question': 'Tell me 3 fruits of my favourite color',
 'chat': "Human: Hello, i lake the orange color.\nAI: Orange color is indeed beautiful! Did you know that data science can also be beautiful in the way it uncovers patterns and insights from complex datasets? Just like how different shades of orange can create a stunning visual impact, the analysis and visualization of data in data science can bring out the beauty of information hidden within the numbers. Let's explore the beauty of data science together!",
 'text': "Sure! The color orange is associated with vibrant and bold fruits such as oranges, apricots, and persimmons. Just like how these fruits bring a burst of color and flavor to your palate, data science can bring a burst of insights and knowledge from analyzing data. The beauty of data science lies in its ability to uncover valuable information and trends that can help drive decision-making and innovation. Let's continue to appreciate the beauty of data science together!"}

In [24]:
print(memory.load_memory_variables({})['chat'])

Human: Hello, i lake the orange color.
AI: Orange color is indeed beautiful! Did you know that data science can also be beautiful in the way it uncovers patterns and insights from complex datasets? Just like how different shades of orange can create a stunning visual impact, the analysis and visualization of data in data science can bring out the beauty of information hidden within the numbers. Let's explore the beauty of data science together!
Human: Tell me 3 fruits of my favourite color
AI: Sure! The color orange is associated with vibrant and bold fruits such as oranges, apricots, and persimmons. Just like how these fruits bring a burst of color and flavor to your palate, data science can bring a burst of insights and knowledge from analyzing data. The beauty of data science lies in its ability to uncover valuable information and trends that can help drive decision-making and innovation. Let's continue to appreciate the beauty of data science together!


## LLM output parsing

<a href="https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/quick_start/" target="_blank">source</a>

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

**Output parsers** are classes that help *structure language model responses*. 

There are two main methods an output parser must implement:

- "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
- "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

- "Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

In [26]:
# OutputParser

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
New question: {question}
Answer:"""

prompt = PromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"question": "Mi piace il colore arancione"})

"Il colore arancione è meraviglioso, così come la bellezza dei dati nel campo della scienza. La data science ci permette di esplorare e analizzare i dati in modi sorprendenti, rivelando nuove informazioni e tendenze che altrimenti potrebbero sfuggire. È affascinante vedere come i dati possano raccontare storie e guidare decisioni importanti. La data science è una vera forma d'arte che porta luce e comprensione nel mondo dei dati."

In [28]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator

class User(BaseModel):
    id: int = Field(description="user identification number")
    name: str = Field(description="user name")
    mail: str = Field(description="user mail address")
    

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=User)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"id": {"title": "Id", "description": "user identification number", "type": "integer"}, "name": {"title": "Name", "description": "user name", "type": "string"}, "mail": {"title": "Mail", "description": "user mail address", "type": "string"}}, "required": ["id", "name", "mail"]}
```


In [29]:
prompt = PromptTemplate(
    template="Analizza il testo\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "user identification number", "type": "integer"}, "name": {"title": "Name", "description": "user name", "type": "string"}, "mail": {"title": "Mail", "description": "user mail address", "type": "string"}}, "required": ["id", "name", "mail"]}\n```'}, template='Analizza il testo\n{format_instructions}\n{query}\n')

In [30]:
chain = prompt | llm | parser

query = "id:1, name: John Doe, e-mail: john.doe@johndoe.com"

chain.invoke({"query": query})

User(id=1, name='John Doe', mail='john.doe@johndoe.com')

In [31]:
chain = prompt | llm | parser

query = "my name is giuseppe mastrandrea, my email is mastrandreagiuseppe@gmail.com and my id is 500"

chain.invoke({"query": query})

User(id=500, name='giuseppe mastrandrea', mail='mastrandreagiuseppe@gmail.com')

In [32]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser(pydantic_object=User)

chain = prompt | llm | json_parser

chain.invoke({"query": query})

{'id': 500,
 'name': 'giuseppe mastrandrea',
 'mail': 'mastrandreagiuseppe@gmail.com'}